In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# Caregando o arquivo
df = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv')
df.head()

In [ ]:
# Selecionando os atributos para a exploração
df.shape[0]
df['LotFrontage'][0]

mydf = df[['LotArea','Street','BldgType','HouseStyle','OverallQual', 'OverallCond','YearBuilt','TotalBsmtSF','BedroomAbvGr','KitchenAbvGr','SalePrice']]
mydf.head()

In [ ]:
# Descrição do conjunto de dados selecionado, nome: mydf
mydf.info()

In [ ]:
# Fazendo a conversão dos atributos qualitativos 
print(mydf['Street'].value_counts())
print(mydf['BldgType'].value_counts())
print(mydf['HouseStyle'].value_counts())
cleanup_nums = {"Street": {"Grvl":0, "Pave":1},
                 "BldgType": {"1Fam":0, "TwnhsE": 1, "Duplex":2, "Twnhs":3, "2fmCon":4},
                 "HouseStyle": {"1Story":0, "2Story":2, "1.5Fin":3, "SLvl":4, "SFoyer":5, "1.5Unf":6, "2.5Unf":7, "2.5Fin":8}}
mydf.replace(cleanup_nums, inplace=True)
print(mydf['Street'].value_counts())
print(mydf['BldgType'].value_counts())
print(mydf['HouseStyle'].value_counts())

In [ ]:
mydf.describe()

In [ ]:
##Wrapper Selection Feature
##coded by jecs89
from sklearn.neural_network import MLPRegressor
import random

def my_wrapper( mydf, N_test, size_col ):

    size_subset = int(mydf.shape[0]/10)
    x_subset = np.zeros([size_subset,size_col+1])

    mydf_data = mydf.get_values()

    vec_row = np.zeros([ N_test, size_subset])
    vec_col = np.zeros([ N_test, size_col ])
    vec_diff = np.zeros([ 1, N_test ])

    for n_test in range(0,N_test):
        x_row = random.sample(range(1, mydf.shape[0]), size_subset)
        x_row = np.sort(x_row)
        vec_row[n_test,:] = x_row

        x_col = random.sample(range(0, mydf_data.shape[1]-1), size_col)
        x_col = np.sort(x_col)
        vec_col[n_test,:] = x_col

        c = 0
        for idj in range(0, mydf_data.shape[1]-1):
            for idx_subset in range(0,size_subset):
                if( idj == x_col[c] ):
                    x_subset[idx_subset,c] = mydf_data[x_row[idx_subset],x_col[c]]
            if(idj == x_col[c]):
                c += 1
            if( c == size_col ):
                break;

        for idx_subset in range(0,size_subset):
            x_subset[idx_subset,x_subset.shape[1]-1] = mydf_data[x_row[idx_subset],mydf_data.shape[1]-1]

        x_st = x_subset[:,0:x_subset.shape[1]-1]
        y_st = x_subset[:,x_subset.shape[1]-1]

        clf = MLPRegressor(hidden_layer_sizes=(100, ), activation='relu', solver='adam', alpha=0.0001).fit(x_st,y_st)
        y_pdt = clf.predict(x_st)
        
        counter = 0
        for i in range(0,x_st.shape[0]):
            counter += (y_st[i] - y_pdt[i])*(y_st[i] - y_pdt[i])

        vec_diff[0,n_test] = counter
#         print(str(clf.score(x_st,y_st))+ ' ' + str(counter))

    my_min = vec_diff[0,0]
    idx = 0
    for i in range(1,N_test):
        if( my_min > vec_diff[0,i]):
            my_min = vec_diff[0,i]
            idx = i
            
#     print(vec_row[idx,:])
#     print(vec_col)
    
    res = np.zeros([mydf_data.shape[0],size_col+1])
    for i in range(0,mydf_data.shape[0]):
        for j in range(0,size_col):
            res[i,j] = mydf_data[i,int(vec_col[idx,j])]
    
    res[:,size_col] = mydf_data[:,mydf_data.shape[1]-1]
    
    return res

In [ ]:
size_col = 3
N_test = 20
res = my_wrapper( mydf=mydf, N_test=N_test, size_col=size_col)
print(res)